# Modelado
### Modelado de la data para modelo de búsqueda funcione correctamente y más rápido

**`Transformaciones`**:  Para este MVP no necesitas transformar los datos dentro del dataset pero trabajaremos en leer el dataset con el formato correcto.


ajustar las fechas

reemplazar &amp

separar generos en columnas

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
rows = []
with open("steam_games.json") as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df = pd.DataFrame(rows)
df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


---
Eliminamos columnas que no vamos a usar para las consultas

In [4]:
df = df.drop(['publisher', 'app_name', 'url', 'tags', 'discount_price', 'reviews_url', 'price', 'id', 'developer'], axis=1)

In [5]:
df[['genres']].head()

,genres
0,"[Action, Casual, Indie, Simulation, Strategy]"
1,"[Free to Play, Indie, RPG, Strategy]"
2,"[Casual, Free to Play, Indie, Simulation, Sports]"
3,"[Action, Adventure, Casual]"
4,NaN


In [6]:
df_fechas_1 = df.copy()
df_fechas_1['release_date'] = pd.to_datetime(df_fechas_1["release_date"], errors='coerce')
df_fechas_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28852 non-null  object        
 1   title         30085 non-null  object        
 2   release_date  29783 non-null  datetime64[ns]
 3   specs         31465 non-null  object        
 4   early_access  32135 non-null  bool          
 5   sentiment     24953 non-null  object        
 6   metascore     2677 non-null   object        
dtypes: bool(1), datetime64[ns](1), object(5)
memory usage: 1.5+ MB


In [7]:
df_fechas = df.copy()

df_fechas['release_date'] = df_fechas['release_date'].str.replace('Coming ', '')

df_fechas['release_date'] = df_fechas['release_date'].str.replace('End', 'Dec')

df_fechas['release_date'] = df_fechas['release_date'].str.replace('Q1', 'Feb')
df_fechas['release_date'] = df_fechas['release_date'].str.replace('Q2', 'May')
df_fechas['release_date'] = df_fechas['release_date'].str.replace('Q3', 'Aug')
df_fechas['release_date'] = df_fechas['release_date'].str.replace('Q4', 'Nov')

df_fechas['release_date'] = df_fechas['release_date'].str.replace('Early', 'Mar')
df_fechas['release_date'] = df_fechas['release_date'].str.replace('Late 2016 - Early 2017', 'Mar')
df_fechas['release_date'] = df_fechas['release_date'].str.replace('Late', 'Nov')

df_fechas['release_date'] = df_fechas['release_date'].str.replace('Spring', 'Apr')
df_fechas['release_date'] = df_fechas['release_date'].str.replace('Summer', 'Jul')
df_fechas['release_date'] = df_fechas['release_date'].str.replace('Autumn', 'Oct')
df_fechas['release_date'] = df_fechas['release_date'].str.replace('Winter', 'Jan')
df_fechas['date1'] = pd.to_datetime(df_fechas["release_date"], format="%b %Y", errors='coerce') # Feb 2017
df_fechas['date2'] = pd.to_datetime(df_fechas["release_date"], format="%B %Y", errors='coerce') # August 2017
df_fechas['release_date'] = pd.to_datetime(df_fechas["release_date"], errors='coerce')

In [8]:
df_fechas['release_date'] = df_fechas['release_date'].fillna(df_fechas['date1'])
df_fechas['release_date'] = df_fechas['release_date'].fillna(df_fechas['date2'])
# df_fechas = df_fechas.drop(['date1', 'date2'], axis=1)
df_fechas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28852 non-null  object        
 1   title         30085 non-null  object        
 2   release_date  29894 non-null  datetime64[ns]
 3   specs         31465 non-null  object        
 4   early_access  32135 non-null  bool          
 5   sentiment     24953 non-null  object        
 6   metascore     2677 non-null   object        
 7   date1         103 non-null    datetime64[ns]
 8   date2         14 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](3), object(5)
memory usage: 2.0+ MB


In [9]:
df_json = df_fechas.copy()
df_json['release_date'] = df_fechas.release_date.dt.strftime('%Y-%m-%d')
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28852 non-null  object        
 1   title         30085 non-null  object        
 2   release_date  29894 non-null  object        
 3   specs         31465 non-null  object        
 4   early_access  32135 non-null  bool          
 5   sentiment     24953 non-null  object        
 6   metascore     2677 non-null   object        
 7   date1         103 non-null    datetime64[ns]
 8   date2         14 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](2), object(6)
memory usage: 2.0+ MB


In [10]:
df_json.to_json('data_search.json')

In [11]:
mask_fechas = df_fechas['release_date'].isnull()
df_fechas_malas = df[mask_fechas]
df_fechas_malas.release_date.unique()

array([nan, 'Soon..', '2017', 'Beta测试已开启', 'Coming Soon', 'TBA',
       "When it's done", 'coming soon', '2018', 'soon', '14 July',
       'To be Announced', 'TBD', 'Fall 2017', 'Январь 2018', 'Soon',
       'First quarter of 2018', 'H2 2018', '21 Jun, 2017', '2018年初頭発売予定',
       'Please wait warmly', 'early access', 'SOON',
       'Coming Soon/Próximamente', '2018 [Now get free Pre Alpha]',
       '1st Quarter 2018', '预热群52756441', '2018年1月', 'Coming soon',
       'Fall 2018',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       '"""Soon"""', 'Demo coming soon.', 'Coming 2017',
       'Not yet available',
       "C'est bientôt...                    (ou pas)",
       'Datachunks conflicted. Be vigiliant.', 'Q2 2018 (Tentative)',
       'Demo is available now!', 'Play Beta in demo!', '2017 Q4',
       'Coming Soon!', 'Early Spring 2018', 'TBA 2017', 'To be announced',
       'When it is finished', 'Release Date TBA', 'When it is re